In [1]:
import numpy as np #matrix math
import math
import tensorflow as tf #machine learning
import matplotlib.pyplot as plt #plotting

%matplotlib inline
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [2]:
# 67 was 128 filters each, including h_dim
# 68 is 128, 64, 32, 16, 16
# 69 inverted
# 70 .. len_edge 128 and filters: chans, 16, 32, 64, 128, 256, 512
attempt=72
num_channels = 4
len_edge = 128
bs = 32
#n_filters = [1, 16, 16, 16, 16, 16]
n_filters = [num_channels, 32, 64, 128, 256, 512, 512] # for mnist
h_dim = 256
latent_dim = 15*15


inedge = len_edge / (2 ** (len(n_filters)-1))
latvisdim = int(np.sqrt(latent_dim))
tb_imgs_to_display = 4
#filename = '/run/media/ron/silver_small/twelve_months/3d/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
#filename = '/home/ron/Documents/local51/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
filename='/media/ramdisk/lil_s2.tif'
#filename = '/run/media/ron/silver_small/twelve_months/3d/S1A_IW_GRDH_1SDV_20160325T083601_20160325T083630_010523_00FA23_6F51.tif'
#filename='/media/ramdisk/smaller.tif'


In [3]:
def sample_a_batch(filebase, batchsize, tilesize=128, sb=0, normalize=False, flattened=True):
    # can sample for spatio-temporal (single_file = False), and spatial-only case (single_file = True).
    import numpy as np
    from osgeo import gdal
    
    if '.' in filebase:
        single_file = True
        S = gdal.Open(filebase)
    else:
        single_file = False
        S = gdal.Open(filebase + '_1.vrt')
        
    samples = []
    
    if single_file:
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1)
            
            #A = np.transpose(S.ReadAsArray(RX[0], RY[0], tilesize, tilesize))           
            
            if sb:
                B = S.GetRasterBand(sb)
                A = np.transpose(B.ReadAsArray(RX[0], RY[0], tilesize, tilesize), dtype=np.float32)                
                #print np.min(A)
                if np.min(A) > 0:
                    if normalize:
                        A = A / normalize[2][sb-1]                        
                    A = np.expand_dims(A, 2)                    
                    if flattened:
                        A = A.flatten()                        
                    samples.append(A)
            else:
                A = np.transpose(S.ReadAsArray(RX[0], RY[0], tilesize, tilesize)).astype(np.float32)
                if np.min(A) > 0:
                    if normalize:
                        for b in range(A.shape[2]):
                            A[:, :, b] = A[:, :, b] / float(normalize[2][b])
                        if flattened:
                            A = A.flatten()
#                        print type(A)
#                        print np.max(A)
                        samples.append(A)
        
    else: # must be overhauled
        while len(samples) < batchsize:
            RX = np.random.randint(S.RasterXSize-tilesize,size=1)
            RY = np.random.randint(S.RasterYSize-tilesize,size=1) 
            
            skip_loc = False
            months = []
            
            for m in range(1,13):
                S = gdal.Open(filebase + '_' + str(m) + '.vrt')
                A = np.transpose(S.ReadAsArray(RX[i], RY[i], tilesize, tilesize))
                if np.min(A) == 0.0:
                    skip_loc = True
                    break
                else:
                    months.append(A)                
            if not skip_loc:
                months = np.array(months)
                samples.append(months)
        
    return np.array(samples)

def normalization_parameters(fn):
    from osgeo import gdal
    S = gdal.Open(fn)
    mns = []
    sds = []
    maxs = []
    
    for b in range(S.RasterCount):
        B = S.GetRasterBand(b+1)
        mn, sd = B.ComputeStatistics(1)[2:4]
        mns.append(mn)
        sds.append(sd)
        maxs.append(B.GetMaximum())
        
    return mns, sds, maxs

In [4]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)
    
def fc_layer(inp, channels_in, channels_out, name='fc'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([channels_in, channels_out]), name='W')
        b = tf.Variable(tf.zeros([channels_out]), name='B')
        return tf.nn.relu(tf.matmul(inp, w) + b)

In [5]:
with tf.name_scope('Input'):
    X = tf.placeholder(tf.float32, shape=([None, len_edge, len_edge, num_channels]))#bs
    if num_channels == 2:
        #X_show = tf.concat([X, X[:, :, :, 0]], axis=3)
        X_show = tf.concat([X, tf.expand_dims(X[:, :, :, 1], 3)], axis=3)
        tf.summary.image('input_images', X_show, max_outputs=tb_imgs_to_display)
    elif num_channels > 3:
        tf.summary.image('input_images', X[:, :, :, 0:3], max_outputs=tb_imgs_to_display)
    else:
        tf.summary.image('input_images', X, max_outputs=tb_imgs_to_display)

cur_input = X

Ws = []    
shapes = []

for l, n_out in enumerate(n_filters[1:]):
    n_input = cur_input.get_shape().as_list()[3]
    shapes.append(cur_input.get_shape().as_list())
    with tf.name_scope('conv_indecon_' + str(l)):
        with tf.variable_scope(str(l)):
            #        W = tf.Variable(initializer=tf.contrib.layers.xavier_initializer_conv2d(), name='weights')
            #W = tf.Variable(tf.random_uniform([3, 3, n_input, n_out], -1.0/math.sqrt(n_input), 1.0/math.sqrt(n_input)), name='weights')
            W = tf.get_variable('weights', [3, 3, n_input, n_out], initializer=tf.contrib.layers.xavier_initializer_conv2d())#, name='weights')
            b = tf.Variable(tf.zeros([n_out]), name='bias')
            tf.summary.histogram('weights', W)
            tf.summary.histogram('biases', b)
            Ws.append(W)
            conv = tf.nn.conv2d(cur_input, W, strides=[1, 2, 2, 1], padding='SAME')
            act = tf.nn.relu(conv+b)#tf.nn.relu(conv,b)
            tf.summary.histogram('activations', act)
        cur_input = act

with tf.name_scope('Dense'):
    with tf.name_scope('Fully_Encode'):
        flattened = tf.reshape(cur_input, [-1, inedge * inedge * n_filters[-1]])# ...
        
#        W_enc = weight_variable([inedge * inedge * n_filters[-1], h_dim], 'W_enc')
        W_enc = tf.get_variable('W_enc', [inedge * inedge * n_filters[-1], h_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_enc = bias_variable([h_dim], 'b_enc')        
        full1 = tf.nn.relu(tf.matmul(flattened, W_enc) + b_enc)
        tf.summary.histogram('weights', W_enc)
        tf.summary.histogram('biases', b_enc)
        tf.summary.histogram('activations', full1)
#        full1 = fc_layer(flattened, 7 * 7 * n_filters[-1], h_dim, 'fc1')
    with tf.name_scope('Mu'):
#        W_mu = weight_variable([h_dim, latent_dim], 'W_mu')
        W_mu = tf.get_variable('W_mu', [h_dim, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
        b_mu = bias_variable([latent_dim], 'b_mu')
        mu = tf.matmul(full1, W_mu) + b_mu
    with tf.name_scope('Logstd'):
        W_logstd = tf.get_variable('W_logstd', [h_dim, latent_dim], initializer=tf.contrib.layers.xavier_initializer())
#        W_logstd = weight_variable([h_dim, latent_dim], 'W_logstd')
        b_logstd = bias_variable([latent_dim], 'b_logstd')
        logstd = tf.matmul(full1, W_logstd) + b_logstd
    with tf.name_scope('VAE_final'):
        noise = tf.random_normal([1, latent_dim])
        z = mu + tf.multiply(noise, tf.exp(.5*logstd))
    with tf.name_scope('Fully_Decode'):
        
        W_dec1 = tf.get_variable('W_dec1', [latent_dim, h_dim], initializer=tf.contrib.layers.xavier_initializer())
#        W_dec1 = weight_variable([latent_dim, h_dim], 'W_dec1')
        b_dec1 = bias_variable([h_dim], 'b_dec1')
        
        full2 = tf.nn.relu(tf.matmul(z, W_dec1) + b_dec1)
        
        tf.summary.histogram('weights_1', W_dec1)
        tf.summary.histogram('biases_1', b_dec1)
        tf.summary.histogram('activations_1', full2)
        
        W_dec2 = tf.get_variable('W_dec2', [h_dim, inedge * inedge * n_filters[-1]], initializer=tf.contrib.layers.xavier_initializer())
#        W_dec2 = weight_variable([h_dim, inedge * inedge * n_filters[-1]], 'W_dec2')
        b_dec2 = bias_variable([inedge * inedge * n_filters[-1]], 'b_dec2')
        
        full3 = tf.nn.relu(tf.matmul(full2, W_dec2) + b_dec2)
        
        tf.summary.histogram('weights_2', W_dec2)
        tf.summary.histogram('biases_2', b_dec2)
        tf.summary.histogram('activations_2', full3)
#        full2 = fc_layer(z, latent_dim, h_dim, 'fc2')
#        full3 = fc_layer(full2, h_dim, 7 * 7 * n_filters[-1], 'fc3')
        reshaped = tf.reshape(full3, [-1, inedge, inedge, n_filters[-1]])
        
z_visual = tf.reshape(z, [-1, latvisdim, latvisdim, 1])

#if num_channels == 2:
##    #z_visual = tf.concat([z_visual, z_visual[:, :, :, 0]], axis=3)
#    z_visual = tf.concat([z_visual, tf.expand_dims(z_visual[:, :, :, 1], 3)], axis=3)
#    tf.summary.image('input_images', z_visual, max_outputs=tb_imgs_to_display)
#elif num_channels > 3:
#    tf.summary.image('input_images', z_visual[:, :, :, :3], max_outputs=tb_imgs_to_display)
#else:
#    tf.summary.image('input_images', z_visual, max_outputs=tb_imgs_to_display)

tf.summary.image('latents', z_visual, max_outputs=tb_imgs_to_display)
tf.summary.histogram('Latent', z)

Ws.reverse()
shapes.reverse()
cur_input = reshaped

for l, shape in enumerate(shapes):
    cur_name = 'deconv' + str(l)  
    W = Ws[l]    
    with tf.name_scope('conv_indecon_' + str(len(Ws)-(l+1))):
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]), name='bias_dec_'+str(l))
        dec = tf.nn.conv2d_transpose(cur_input, W, tf.stack([bs, shape[1], shape[2], shape[3]]), strides=[1,2,2,1], padding='SAME')
        if l+1 < len(shapes):
            act = tf.nn.relu(dec+b)
            cur_input = act

with tf.name_scope('reconst'):
    reconstruction = tf.nn.sigmoid(dec + b)
    if num_channels == 2:
        r_show = tf.concat([reconstruction, tf.expand_dims(reconstruction[:, :, :, 1], 3)], axis=3)
        tf.summary.image('reconstructed_images', r_show, max_outputs=tb_imgs_to_display)
    elif num_channels > 3:
        tf.summary.image('reconstructed_images', reconstruction[:, :, :, 0:3], max_outputs=tb_imgs_to_display)
    else:
        tf.summary.image('reconstructed_images', reconstruction, max_outputs=tb_imgs_to_display)
#    tf.summary.image('reconstructed_images', reconstruction, max_outputs=tb_imgs_to_display)http://localhost:8888/notebooks/Documents/local51/vae/conv_vae.ipynb#

In [6]:
X_flat = tf.contrib.layers.flatten(X)
R_flat = tf.contrib.layers.flatten(reconstruction)

log_likelihood = tf.reduce_sum(X_flat*tf.log(R_flat + 1e-9)+(1 - X_flat)*tf.log(1 - R_flat + 1e-9), reduction_indices=1)
tf.summary.scalar('LogLike', tf.reduce_mean(log_likelihood))

KL_term = -.5*tf.reduce_sum(1 + 2*logstd - tf.pow(mu,2) - tf.exp(2*logstd), reduction_indices=1)
tf.summary.scalar('KL', tf.reduce_mean(KL_term))

variational_lower_bound = tf.reduce_mean(log_likelihood - KL_term)
tf.summary.scalar('cost', variational_lower_bound)

optimizer = tf.train.AdamOptimizer(1e-4).minimize(-variational_lower_bound)

In [7]:
merged_summary = tf.summary.merge_all()
init = tf.global_variables_initializer()
#init = tf.contrib.layers.xavier_initializer_conv2d()
sess = tf.InteractiveSession()
sess.run(init)
saver = tf.train.Saver()
writer = tf.summary.FileWriter('./vae_logs/' + str(attempt))
writer.add_graph(sess.graph)
stats = normalization_parameters(filename)

In [ ]:
num_iterations = 1000000
recording_interval = 100
#variational_lower_bound_array = []
#log_likelihood_array = []
#KL_term_array = []
#iteration_array = [i*recording_interval for i in range(num_iterations/recording_interval)]

for i in range(num_iterations):
#    mn_l = mnist.train.next_batch(bs)[0]
    x_batch = cifars[i*bs:(i+1)*bs, :, :, :]
#    x_batch = np.reshape(mn_l, [bs, len_edge, len_edge, 1]) # for mnist
#    x_batch = sample_a_batch(filename, bs, len_edge, sb=None, normalize=stats, flattened=False)
    _, s = sess.run([optimizer, merged_summary], feed_dict={X: x_batch})  
#    print i
    if (i%recording_interval == 0):
#        print i
        writer.add_summary(s, i)
        #every 1K iterations record these values
#        vlb_eval = variational_lower_bound.eval(feed_dict={X: x_batch})
#        print "Iteration: {}, Loss: {}".format(i, vlb_eval)

#i=0
#while True:
#    ba = np.random.randint(49500):
#    x_batch = np.expand_dims(cifars[ba:ba+bs, :, :, 0], 3)
#    _, s = sess.run([optimizer, merged_summary], feed_dict={X: x_batch})
#    i += 1
#    if (i%recording_interval == 0):
#        print i
#        writer.add_summary(s,i)

In [17]:
X_rsbig = sample_a_batch(filename, 320, len_edge, sb=None, normalize=stats, flattened=False)

In [21]:
ep = 0

In [28]:
#i=0
recording_interval = 20
update_interval = 50000
while True:
    ep += 32
#    ba = np.random.randint(450)
#    x_batch = cifars[ba:ba+bs, :, :, :]
#    x_batch = X_rs[ba:ba+bs, :, :, :]
#    x_batch = X_rsbig[ep-32:ep]
    x_batch = X_rs1[:bs, :, :, :]
    _, s = sess.run([optimizer, merged_summary], feed_dict={X: x_batch})
    i += 1
    if (i%recording_interval == 0):
        print i
        writer.add_summary(s,i)
    if ep == 320:
        ep = 0
        
#        if (i%update_interval == 0):
#            X_rsm = sample_a_batch(filename, bs, len_edge, sb=None, normalize=stats, flattened=False)

384460
384480
384500
384520
384540
384560
384580
384600
384620
384640
384660
384680
384700
384720
384740
384760
384780
384800
384820


KeyboardInterrupt: 

In [25]:
x_batch.shape

(32, 128, 128, 4)

**IGNORE FOLLOWING CELLS:**

In [ ]:


def conv_layer(inp, channels_in, channels_out, vscope, name='conv'):
    with tf.name_scope(name):
        with tf.variable_scopec(vscope):
            tf.Variable(tf.zeros([3, 3, channels_in, channels_out]), name='W')
        b = tf.Variable(tf.zeros([channels_out]), name='B')
        conv = tf.nn.conv2d(inp, w, strides=[1, 1, 1, 1], padding='SAME')
        act = tf.nn.relu(conv + b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activations', act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
def conv_t_layer(inp, channels_in, channels_out, name='conv_t'):#, activation=tf.nn.relu):
    with tf.name_scope(name):
        #w = tf.Variable(tf.zeros([3, 3, channels_out, channels_in]), name='W')
        w = conv1
        b = tf.Variable(tf.zeros([conv1.get_shape().as_list()[2]]))
        
        tf.nn.conv2d_transpose(inp, W, tf.stack([]))
        
        
        
        b = tf.Variable(tf.zeros([channels_out]), name='B')
        batch_size = 32 #tf.shape(inp)[0]
        deconv_shape = tf.stack([batch_size, inp.shape[2].value * 2, inp.shape[2].value * 2, channels_out])
        conv_t = tf.nn.conv2d_transpose(inp, w, deconv_shape, strides=[1, 2, 2, 1], padding='SAME')
        #act = activation(conv_t + b)
        act = tf.nn.relu(conv_t + b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activations', act)
        return act#tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'

In [ ]:
with tf.name_scope('Input'):
    X = tf.placeholder(tf.float32, shape=([None, len_edge, len_edge, num_channels]))#bs
    tf.summary.image('input_images', X, max_outputs=3)

conv1 = conv_layer(X, num_channels, filters[0], 'conv1')
conv2 = conv_layer(conv1, conv1.shape[-1].value, filters[1], 'conv2')
conv3 = conv_layer(conv2, conv2.shape[-1].value, filters[2], 'conv3')
conv4 = conv_layer(conv3, conv3.shape[-1].value, filters[3], 'conv4')

#with tf.name_scope('Dense_Encode'):
flattened = tf.reshape(conv4, [-1, 2 * 2 * filters[-1]])
full1 = fc_layer(flattened, 2 * 2 * filters[-1], h_dim, 'fc1') # channels_in ?????

W_mu = weight_variable([h_dim, latent_dim], 'W_mu')
b_mu = bias_variable([latent_dim], 'b_mu')
mu = FC_layer(full1, W_mu, b_mu)
W_logstd = weight_variable([h_dim, latent_dim], 'W_logstd')
b_logstd = bias_variable([latent_dim], 'b_logstd')
logstd = FC_layer(full1, W_logstd, b_logstd)

noise = tf.random_normal([1, latent_dim])
z = mu + tf.multiply(noise, tf.exp(.5*logstd))

z_visual = tf.reshape(z, [-1, 6, 6, 1])
tf.summary.image('latents', z_visual, max_outputs=3)

tf.summary.histogram('Latent', z)

full2 = fc_layer(z, latent_dim, h_dim, 'fc2')
full3 = fc_layer(full2, h_dim, 2 * 2 * filters[-1], 'fc3')# ???????

reshaped = tf.reshape(full3, [-1, 2, 2, filters[-1]])

conv_t1 = conv_t_layer(reshaped, filters[1], filters[1], 'conv_t1')
conv_t2 = conv_t_layer(conv_t1, filters[1], filters[1], 'conv_t2')
conv_t3 = conv_t_layer(conv_t2, filters[1], filters[1], 'conv_t3')
reconstruction = conv_t_layer(conv_t3, filters[1], 1, 'conv_t4')#, tf.nn.sigmoid)

tf.summary.image('reconstructed_images', reconstruction, max_outputs=3) #............

In [ ]:
import pickle
cifars = []
for i in range(1, 6):
    with open('/home/ron/Downloads/cifar-10-batches-py/data_batch_' + str(i), mode='rb') as f:
        data = pickle.load(f)
        raw_images = data[b'data']
        cls = np.array(data[b'labels'])        
        raw_float = np.array(raw_images, dtype=float) / 255.0
        images = raw_float.reshape([-1, 3, len_edge, len_edge])
        images = images.transpose([0, 2, 3, 1])        
        cifars = cifars + list(images)        
cifars = np.array(cifars)

In [ ]:
x_batch.shape

In [ ]:
cifars.shape

In [ ]:
50000/32